In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Exercises") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/19 16:54:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/19 16:54:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/19 16:54:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [185]:
event_table = spark.read.csv('EventTable.csv')
user_table = spark.read.csv('UserTable.csv')
video_table = spark.read.csv('VideoTable.csv')
series_table = spark.read.csv('SeriesTable.csv')

In [186]:
print("TS UserID ItemID")
event_table.show()
print("UserID RegDate BirthDate SubscrTag")
user_table.show()
print("ItemID Genre LengthSec")
video_table.show()
print("ItemID SeriesID SeriesName")
series_table.show()

TS UserID ItemID
+-----+---+---+
|  _c0|_c1|_c2|
+-----+---+---+
|12:15|  1|  1|
|12:20|  1|  2|
|12:30|  2|  1|
|12:40|  2|  2|
|12:50|  1|  3|
+-----+---+---+

UserID RegDate BirthDate SubscrTag
+---+-----------+-----------+---------+
|_c0|        _c1|        _c2|      _c3|
+---+-----------+-----------+---------+
|  1| 12.11.2001| 10.10.1990|   Active|
|  2| 10.06.2001| 05.05.1991| ChurnOut|
+---+-----------+-----------+---------+

ItemID Genre LengthSec
+---+------+---+
|_c0|   _c1|_c2|
+---+------+---+
|  1|Comedy|100|
|  2|Action|200|
|  3|Comedy|300|
+---+------+---+

ItemID SeriesID SeriesName
+---+---+-------+
|_c0|_c1|    _c2|
+---+---+-------+
|  1| 10|Series0|
|  2| 11|Series1|
+---+---+-------+



In [39]:
df.agg({"_c2": "max"}).show()

+--------+
|max(_c2)|
+--------+
|       2|
+--------+



In [40]:
df.agg({"_c2": "min"}).show()

+--------+
|min(_c2)|
+--------+
|       1|
+--------+



In [41]:
df.agg({"_c2": "avg"}).show()

+--------+
|avg(_c2)|
+--------+
|     1.5|
+--------+



In [130]:
event_table_rdd = event_table.rdd

In [175]:
event_table_rdd.first()

Row(_c0='12:15', _c1='1', _c2='1')

In [174]:
event_table.groupBy("_c1").count()

DataFrame[_c1: string, count: bigint]

In [176]:
# lambda row: row.get("McID")
from operator import add
event_table_rdd\
    .map(lambda row: (int(row["_c1"]), int(row["_c2"])))\
    .reduceByKey(lambda x, y: y)\
    .collect()

[(1, 2), (2, 2)]

In [59]:
print(event_table_rdd.collect())

[Row(_c0='12:15', _c1='1', _c2='1'), Row(_c0='12:20', _c1='1', _c2='2'), Row(_c0='12:30', _c1='2', _c2='1'), Row(_c0='12:40', _c1='2', _c2='2')]


In [60]:
event_table_rdd.reduce(lambda a, b: 2 * b)

('12:40', '2', '2', '12:40', '2', '2')

In [189]:
from pyspark.sql.functions import avg, sum

In [199]:
# sales_table.join(products_table, sales_table["product_id"] == products_table["product_id"], "inner")\
#    .agg(avg(products_table["price"] * sales_table["num_pieces_sold"])).show()
event_table.join(video_table, event_table["_c2"] == video_table["_c0"], "inner").show()
    #.agg(avg(video_table["_c2"])).show()

+-----+---+---+---+------+---+
|  _c0|_c1|_c2|_c0|   _c1|_c2|
+-----+---+---+---+------+---+
|12:15|  1|  1|  1|Comedy|100|
|12:20|  1|  2|  2|Action|200|
|12:30|  2|  1|  1|Comedy|100|
|12:40|  2|  2|  2|Action|200|
|12:50|  1|  3|  3|Comedy|300|
+-----+---+---+---+------+---+



In [200]:
user_table_rdd = user_table.rdd

In [262]:
def reduceFunction(elem):  
    if elem[1] == " ChurnOut":
        return elem[0]

user_table_rdd\
    .map(lambda x: (x["_c0"], x["_c3"]))\
    .map(lambda x: reduceFunction(x)).collect()

[None, '2']